# 9. Dueling DQN
- DQN 算法敲开了深度强化学习的大门，但是作为先驱性的工作，其本身存在着一些问题以及一些可以改进的地方。于是，在 DQN 之后，学术界涌现出了非常多的改进算法...

其中有两个非常著名的算法：**Double DQN 和 Dueling DQN**
本节学习 **Dueling DQN**
(更多、更详细的 DQN 改进方法，见 Rainbow 模型的论文及其引用文献:[0]Rainbow: Combining improvements in deep reinforcement learning)

## 9.1 Dueling DQN 原理
根据前期的学习可知:在同一状态下,**所有动作**的动作价值的期望就是**该状态**的状态价值
那么完全可以将 **状态价值** 剥离出来
将 **状态动作价值函数Q** 减去 **状态价值函数V** 的结果定义为 **优势函数A** :$A(s,a)=Q(s,a)-V(s)$
该函数可以独自表达各个动作在同一状态下的 **差异**，即衡量在状态 s 下选择动作 a 相对于平均水平的好坏

实际上原始的DQN算法Q值也有大小之分,那么为何要多此一举定义一个独自表达差异的函数呢?
本质上是要提升网络学习效率和泛化能力:
- 不是所有状态都需要区分动作(核心)：如在 Atari 游戏中，角色正处于安全区，向左/向右/跳跃都不会立即影响得分，但原始 DQN 仍要为每个动作分别拟合 Q 值
- 更结构化，对动作多差异小的状态更有优势：分离状态与差异,根据实际情况有选择的关注(当智能体前面没有车时，车辆自身动作并没有太大差异，此时智能体更关注状态价值；而当智能体前面有车时（智能体需要超车），智能体开始关注不同动作优势值的差异)

![状态价值和优势值的简单例子](Illustrations/状态价值和优势值的简单例子.png)

在 Dueling DQN 中，Q 网络被重建为：
$$Q_{\eta,\alpha,\beta}(s,a)=V_{\eta,\alpha}(s)+A_{\eta,\beta}(s,a)$$
#### 参数说明:
$\eta$(整个函数近似器（神经网络）的共享参数)：在 Dueling DQN 网络中，该参数是整个函数近似器（神经网络）的共享参数，用于前面几层提取共享状态的表示
$\alpha$：专用于拟合状态价值函数的网络分支的参数，而该网络从共享特征中抽取更专注于“状态本身”的信息
$\beta$：专用于拟合优势函数的网络分支的参数，该网络主要用于捕捉特定动作的相对优势

以下为网络结构对比图：
![DQN与Dueling%20DQN网络结构](Illustrations/DQN与Dueling%20DQN网络结构.jpg)

## 9.2 Dueling DQN 代码实现

导入相关库：

In [1]:
from utils.replay_buffer import ReplayBuffer
from utils.train import train_on_policy_agent, train_off_policy_agent
from utils.advantage import compute_advantage
from utils.smoothing import moving_average